# Ray RLlib - Sample Application: CartPole-v0

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

We were briefly introduced to the _CartPole_ example and the OpenAI gym `CartPole-v0` environment ([gym.openai.com/envs/CartPole-v0/](https://gym.openai.com/envs/CartPole-v0/)) in the [introduction](01-Introduction-to-Reinforcement-Learning.ipynb). This lesson uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to train a policy for _CartPole_.

Recall that the `gym` Python module provides MDP interfaces to a variety of simulators, like the simple simulator for the physics of balancing a pole on a cart that is used by the CartPole environment. The _CartPole_ problem is described at https://gym.openai.com/envs/CartPole-v0. Here is an image from that website:

![Cart Pole](../images/rllib/Cart-Pole.png)

Even though this is a relatively simple and quick example to run, its results can be understood quite visually.

For more background about this problem, see:

* ["Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem"](https://ieeexplore.ieee.org/document/6313077), AG Barto, RS Sutton, and CW Anderson, *IEEE Transactions on Systems, Man, and Cybernetics* (1983)
* ["Cartpole - Introduction to Reinforcement Learning (DQN - Deep Q-Learning)"](https://towardsdatascience.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288), [Greg Surma](https://twitter.com/GSurma)

First, import Ray and the PPO module in RLlib, then start Ray.

In [1]:
import ray
import ray.rllib.agents.ppo as ppo

In [2]:
import pandas as pd
import json, os, shutil, sys

In [3]:
sys.path.append('..') # so we can import from "util"
from util.line_plots import plot_line, plot_line_with_min_max, plot_line_with_stddev

Model *checkpoints* will get saved after each iteration into directories under `tmp/ppo/cart`, i.e., relative to this directory. 
The default directories for checkpoints are `$HOME/ray_results/<algo_env>/.../checkpoint_N`.

> **Note:** If you prefer to use a different directory root, change it in the next cell _and_ in the `rllib rollout` command below.

In [4]:
checkpoint_root = 'tmp/ppo/cart'

Clean up output of previous lessons (optional):

In [5]:
# Where checkpoints are written:
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)

# Where some data will be written and used by Tensorboard below:
ray_results = f'{os.getenv("HOME")}/ray_results/'
shutil.rmtree(ray_results, ignore_errors=True, onerror=None)

Make sure Ray is running:

In [6]:
!../tools/start-ray.sh  --check --verbose

INFO: Ray is already running.


In [7]:
ray.init(ignore_reinit_error=True)

2020-06-13 10:57:41,395	INFO resource_spec.py:212 -- Starting Ray with 4.0 GiB memory available for workers and up to 2.01 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-13 10:57:41,777	INFO services.py:1170 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.1.149',
 'raylet_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:37058',
 'object_store_address': '/tmp/ray/session_2020-06-13_10-57-41_384974_78799/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-13_10-57-41_384974_78799/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-06-13_10-57-41_384974_78799'}

The Ray Dashboard is useful for monitoring Ray:

In [8]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Dashboard URL: http://localhost:8266


Next we'll train an RLlib policy with the [`CartPole-v0` environment](https://gym.openai.com/envs/CartPole-v0/).

If you've gone through the _Multi-Armed Bandits_ lessons, you may recall that we used [Ray Tune](http://tune.io), the Ray Hyperparameter Tuning system, to drive training. Here we'll do it ourselves.

By default, training runs for `10` iterations. Increase the `N_ITER` setting if you want to train longer and see the resulting rewards improve. However, if the max score of `200` is achieved early, you can use a smaller number of iterations.


- `num_workers` is the number of actors that the agent will create. This determines the degree of parallelism that will be used. In a cluster, these actors will be spread over the available nodes.
- `num_sgd_iter` is the number of epochs of SGD (stochastic gradient descent, i.e., passes through the data) that will be used to optimize the PPO surrogate objective at each iteration of PPO, for each _minibatch_ ("chunk") of training data. Using minibatches is more efficient than training with one record at a time.
- `sgd_minibatch_size` is the SGD minibatch size (batches of data) that will be used to optimize the PPO surrogate objective.
- `model` contains a dictionary of parameters describing the neural net used to parameterize the policy. The `fcnet_hiddens` parameter is a list of the sizes of the hidden layers. Here, we have two hidden layers of size 100, each.
- `num_cpus_per_worker` when set to 0 prevents Ray from pinning a CPU core to each worker, which means we could run out of workers in a constrained environment like a laptop or a cloud VM.

> **Note:** If you change the values shown for `config['model']['fcnet_hiddens']`, make the same change in the `rllib rollout` command below!

In [9]:
SELECT_ENV = "CartPole-v0"                      # Specifies the OpenAI Gym environment for Cart Pole
N_ITER = 10                                     # Number of training runs.

config = ppo.DEFAULT_CONFIG.copy()              # PPO's default configuration. See the next code cell.
config["log_level"] = "WARN"                    # Suppress too many messages, but try "INFO" to see what can be printed.
# Other settings we might adjust:
config['num_workers'] = 1                       # Use > 1 for cluster training
config['num_sgd_iter'] = 10                     # Number of SGD (stochastic gradient descent) iterations per training minibatch.
                                                # I.e., for each minibatch of data, do this many passes over it to train. 
config['sgd_minibatch_size'] = 250              # The amount of data records per minibatch
config['model']['fcnet_hiddens'] = [100, 50]    #
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed

Out of curiousity, let's see what configuration settings are defined for PPO. Note in particular the parameters for the deep learning `model`:

In [10]:
ppo.DEFAULT_CONFIG

{'num_workers': 2,
 'num_envs_per_worker': 1,
 'rollout_fragment_length': 200,
 'sample_batch_size': -1,
 'batch_mode': 'truncate_episodes',
 'num_gpus': 0,
 'train_batch_size': 4000,
 'model': {'conv_filters': None,
  'conv_activation': 'relu',
  'fcnet_activation': 'tanh',
  'fcnet_hiddens': [100, 50],
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action_reward': False,
  'state_shape': None,
  'framestack': True,
  'dim': 84,
  'grayscale': False,
  'zero_mean': True,
  'custom_model': None,
  'custom_action_dist': None,
  'custom_options': {},
  'custom_preprocessor': None},
 'optimizer': {},
 'gamma': 0.99,
 'horizon': None,
 'soft_horizon': False,
 'no_done_at_end': False,
 'env_config': {},
 'env': None,
 'normalize_actions': False,
 'clip_rewards': None,
 'clip_actions': True,
 'preprocessor_pref': 'deepmind',
 'lr': 5e-05,
 'monitor': False,
 'log_level': 'WARN

In [13]:
agent = ppo.PPOTrainer(config, env=SELECT_ENV)

results = []
episode_data = []
episode_json = []
for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']}
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    print(f'{n:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}. Checkpoint saved to {file_name}')

2020-06-13 10:58:57,272	INFO trainable.py:217 -- Getting current IP.
2020-06-13 10:58:57,273	WARNING util.py:37 -- Install gputil for GPU system monitoring.


  0: Min/Mean/Max reward:   9.0000/ 22.3820/ 88.0000. Checkpoint saved to tmp/ppo/cart/checkpoint_1/checkpoint-1
  1: Min/Mean/Max reward:  10.0000/ 31.5591/121.0000. Checkpoint saved to tmp/ppo/cart/checkpoint_2/checkpoint-2
  2: Min/Mean/Max reward:  11.0000/ 46.2700/143.0000. Checkpoint saved to tmp/ppo/cart/checkpoint_3/checkpoint-3
  3: Min/Mean/Max reward:  12.0000/ 67.1600/200.0000. Checkpoint saved to tmp/ppo/cart/checkpoint_4/checkpoint-4
  4: Min/Mean/Max reward:  12.0000/ 91.7100/200.0000. Checkpoint saved to tmp/ppo/cart/checkpoint_5/checkpoint-5
  5: Min/Mean/Max reward:  14.0000/119.1200/200.0000. Checkpoint saved to tmp/ppo/cart/checkpoint_6/checkpoint-6
  6: Min/Mean/Max reward:  22.0000/140.0300/200.0000. Checkpoint saved to tmp/ppo/cart/checkpoint_7/checkpoint-7
  7: Min/Mean/Max reward:  22.0000/159.1400/200.0000. Checkpoint saved to tmp/ppo/cart/checkpoint_8/checkpoint-8
  8: Min/Mean/Max reward:  44.0000/166.6100/200.0000. Checkpoint saved to tmp/ppo/cart/checkpoin

The episode rewards should increase after multiple iterations. Try tweaking the config parameters. Smaller values for the `num_sgd_iter`, `sgd_minibatch_size`, or the `model`'s `fcnet_hiddens` will train faster, but take longer to improve the policy.

In [14]:
df = pd.DataFrame(data=episode_data)
df

,n,episode_reward_min,episode_reward_mean,episode_reward_max,episode_len_mean
0,0,9.0,22.382022,88.0,22.382022
1,1,10.0,31.559055,121.0,31.559055
2,2,11.0,46.270000,143.0,46.270000
3,3,12.0,67.160000,200.0,67.160000
4,4,12.0,91.710000,200.0,91.710000
5,5,14.0,119.120000,200.0,119.120000
6,6,22.0,140.030000,200.0,140.030000
7,7,22.0,159.140000,200.0,159.140000
8,8,44.0,166.610000,200.0,166.610000
9,9,44.0,172.290000,200.0,172.290000


In [15]:
import bokeh.io
# The next two lines prevent Bokeh from opening the graph in a new window.
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [17]:
plot_line_with_min_max(df, x_col='n', y_col='episode_reward_mean', min_col='episode_reward_min', max_col='episode_reward_max',
                       title='Cart Pole Episode Rewards', x_axis_label = 'n', y_axis_label='reward')

([image](../images/rllib/Cart-Pole-Episode-Rewards3.png))

Also, print out the policy and model to see the results of training in detail…

In [18]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

[<tf.Variable 'default_policy/fc_1/kernel:0' shape=(4, 100) dtype=float32>,
 <tf.Variable 'default_policy/fc_1/bias:0' shape=(100,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/kernel:0' shape=(4, 100) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_1/bias:0' shape=(100,) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/kernel:0' shape=(100, 50) dtype=float32>,
 <tf.Variable 'default_policy/fc_2/bias:0' shape=(50,) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/kernel:0' shape=(100, 50) dtype=float32>,
 <tf.Variable 'default_policy/fc_value_2/bias:0' shape=(50,) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/kernel:0' shape=(50, 2) dtype=float32>,
 <tf.Variable 'default_policy/fc_out/bias:0' shape=(2,) dtype=float32>,
 <tf.Variable 'default_policy/value_out/kernel:0' shape=(50, 1) dtype=float32>,
 <tf.Variable 'default_policy/value_out/bias:0' shape=(1,) dtype=float32>]
<tf.Tensor 'Reshape:0' shape=(?,) dtype=float32>
Model: "model"
________________

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies), using the `rllib rollout` command line, to evaluate the trained policy.

This visualizes the "cartpole" agent operating within the simulation: moving the cart left or right to avoid having the pole fall over.

We'll use the seventh saved checkpoint, `checkpoint_7` for the rollout, evaluated through `2000` steps.

> **Note:** If you defined `checkpoint_root` above to be different than `tmp/ppo/cart`, then change it here, too. Note that bugs in variable substitution in Jupyter notebooks, we can't use variables in the next cell, unfortunately.

Modify the path to view other checkpoints. Note that you have to change the number _twice_ in the path. You will have to change the checkpoint choice if you changed `N_ITER` above to a smaller number.

> **Note:** If you changed the model parameters, specifically the `fcnet_hiddens` array in the `config` object above, make the same change here.

You may need to make one more modification, depending on how you are running this tutorial:

1. Running on your laptop? - Remove the line `--no-render`. 
2. Running on the Anyscale Service? The pop windows that would normally be created by the rollout can't be seen. Hence, the `--no-render` flag suppresses them. The following cell provides a sample video. 

In [28]:
!RAY_ADDRESS=auto rllib rollout tmp/ppo/cart/checkpoint_7/checkpoint-7 \
    --config "{\"env\": \"CartPole-v0\", \"model\": {\"fcnet_hiddens\": [100, 50]}}" \
    --run PPO \
    --no-render \
    --steps 2000

2020-06-13 11:50:52,188	WARNING worker.py:809 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.
2020-06-13 11:50:52,216	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-06-13 11:50:52,234	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2020-06-13 11:50:54,380	INFO trainable.py:217 -- Getting current IP.
2020-06-13 11:50:54,380	WARNING util.py:37 -- Install gputil for GPU system monitoring.
2020-06-13 11:50:54,438	INFO trainable.py:217 -- Getting current IP.
2020-06-13 11:50:54,438	INFO trainable.py:423 -- Restored on 192.168.1.149 from checkpoint: tmp/ppo/cart/checkpoint_7/checkpoint-7
2020-06-13 11:50:54,438	INFO trainable.py:430 -- Current state after restoring: {'_iteration': 7, '_timesteps_total': 28000, '_time_total': 27.542689323425293, '_episodes_total': 508}
Episode #0: reward: 200.

Here is a sample episode. 

> **Note:** This video was created by running the previous `rllib rollout` command with the argument `--video-dir some_directory`. It creates one video per episode.

In [29]:
from IPython.display import Video

cart_pole_sample_video='../images/rllib/Cart-Pole-Example-Video.mp4'
Video(cart_pole_sample_video)

Finally, launch [TensorBoard](https://ray.readthedocs.io/en/latest/rllib-training.html#getting-started) in a terminal window and open the URL printed to the output. Select the Cart Pole runs and visualize the key metrics from training with RLlib.

```shell
tensorboard --logdir=$HOME/ray_results
```

Next, go through any of the following lessons:

* [04a: Application: Mountain Car](04a-Application-Mountain-Car.ipynb) -- Based on the `MountainCar-v0` environment from OpenAI Gym.
* [04b: Application: Taxi](04b-Application-Taxi.ipynb) -- Based on the `Taxi-v3` environment from OpenAI Gym.
* [04c: Application: Frozen Lake](04c-Application-Frozen-Lake.ipynb) -- Based on the `FrozenLake-v0` environment from OpenAI Gym.

Use TensorBoard to visualize their training runs, metrics, etc., as well. (These notebooks won't mention this suggestion.)